In [1]:
import yaml
import os
from pathlib import Path
import pandas as pd
import struct


# Setup

In [2]:
# experiment setup
PDK = 'sky130'
CLOCK_PERIOD = 10 if PDK == 'sky130' else 2 # ns

# useful paths
energy_char_dpath = Path(os.getcwd()).parent
tests_dpath = energy_char_dpath/f'experiments/tests-{PDK}'
tests_dpath

PosixPath('/bwrcq/scratch/nayiri/hammer-sep24/e2e/hammer-energy-char/experiments/tests-sky130')

# Generate experiment files

In [3]:
N_ITER = 50

max4 = (1 << 4) - 1
max8 = (1 << 8) - 1
max32 = (1 << 32) - 1
max64 = (1 << 64) - 1


# tests dict
#   name format: <design>-<test_name>
#       inputs: list, where each item is a tuple of items per line in input.txt
#       defines: for verilog

## add

In [4]:
adder_widths = [8,32] # [8,16,32,64]

In [5]:
add_tests = {}

for w in adder_widths:
    design = f"add{w}"
    maxw = (1 << w) - 1
    max0 = int(maxw/2)
    max1 = maxw - max0
    d = {
        f'{design}-zero': {
            'inputs': [(0,0) for _ in range(N_ITER)],
            'defines': dict(WIDTH=w),
        },
        f'{design}-max_input_switching': {
            # input operands alternate between 0 and all 1's
            'inputs': [((0,0) if i%2 else (maxw,maxw)) for i in range(N_ITER)],
            'defines': dict(WIDTH=w),
        },
        f'{design}-max_output_switching': {
            # output operands alternate between 0 and all 1's (so set inputs accordingly)
            'inputs': [((0,0) if i%2 else (max0,max1)) for i in range(N_ITER)],
            'defines': dict(WIDTH=w),
        },
    }
    add_tests.update(d)

for t in add_tests:
    add_tests[t]['design'] = t.split('-')[0]
    add_tests[t]['inst'] = '/add/adder0'
    add_tests[t]['clock'] = 'clock'
    add_tests[t]['vsrcs'] = ['src/add.v']
    add_tests[t]['vsrcs_tb'] = ['src/add_tb.v']
    add_tests[t]['top_module'] = 'add'
    add_tests[t]['tb_name'] = 'add_tb'
    add_tests[t]['tb_dut'] = 'add_dut'
    add_tests[t]['input_ports'] = ['in0','in1']
    add_tests[t]['output_ports'] = ['out']


## all

Combine all test dicts

In [6]:
tests_dict = add_tests

### Input files

In [7]:
# create dirs
for t,td in tests_dict.items():
    # experiment directory - will contain all input/output files
    root = tests_dpath/t
    root.mkdir(exist_ok=True,parents=True)
    td['defines']['TESTROOT'] = root
    td['root'] = root
    # hammer build directory
    td['obj_dpath'] = energy_char_dpath/f"build-{PDK}-cm/{td['design']}"

# convert data operands to binary format to dump to input.txt (gets more complicated for floats)
def val2binary(val,input_format='') -> str:
    if type(val) == str: return val
    elif type(val) == int: return '{0:b}'.format(val)
    elif type(val) == float:
        pack_format = '!e'
        if input_format == 'float32':       pack_format = '!f'
        elif input_format == 'float64':     pack_format = '!d'
        return ''.join('{:0>8b}'.format(c) for c in struct.pack(pack_format, val))
    else: assert(False), f"Invalid dtype, {type(val)}"


# write out input.txt
for test,td in tests_dict.items():
    input_format = td['input_format'] if 'input_format' in td else '' # only used for floating point
    with (td['root']/'input.txt').open('w') as f:
        for operands in td['inputs']:
            f.write(" ".join([val2binary(operand,input_format) for operand in operands]) + '\n')


### Hammer Config

In [8]:
def write_cfg(td):
  defines_str = '\n'.join( [ f"  - {key}={val}" for key,val in td['defines'].items() ] )
  clock_period = 10 if PDK == 'sky130' else 2
  delays = [f"""{{name: {i}, clock: {td['clock']}, delay: "1", direction: input}}""" for i in td['input_ports']]
  delays += [f"""{{name: {i}, clock: {td['clock']}, delay: "1", direction: output}}""" for i in td['output_ports']]
  delays = ',\n  '.join(delays)
  cfg = f"""\
vlsi.core.build_system: make
vlsi.inputs.power_spec_type: cpf
vlsi.inputs.power_spec_mode: auto

design.defines: &DEFINES
  - CLOCK_PERIOD={clock_period}
{defines_str}

vlsi.inputs.clocks: [{{name: "clock", period: "{clock_period}ns", uncertainty: "100ps"}}]

vlsi.inputs.delays: [
  {delays}
]

synthesis.inputs:
  top_module: {td['top_module']}
  input_files: {td['vsrcs']}
  defines: *DEFINES

sim.inputs:
  top_module: {td['top_module']}
  tb_name: {td['tb_name']}
  tb_dut: {td['tb_dut']}
  options: ["-timescale=1ns/10ps", "-sverilog"]
  options_meta: append
  defines: *DEFINES
  defines_meta: append
  level: rtl
  input_files: {td['vsrcs'] + td['vsrcs_tb']}

vlsi.core.power_tool: hammer.power.joules
power.inputs:
  level: rtl
  top_module: {td['top_module']}
  tb_name: {td['tb_name']}
  tb_dut: {td['tb_dut']}
  defines: *DEFINES
  input_files: {td['vsrcs']}
  report_configs:
    - waveform_path: {td['root']}/output.fsdb
      report_stem: {td['root']}/power
      toggle_signal: {td['clock']}
      num_toggles: 1
      levels: all
      output_formats:
      - report
      - plot_profile
"""
  with (td['root']/'config.yml').open('w') as f:
    f.write(cfg)

for t in tests_dict:
  write_cfg(tests_dict[t])

## Run experiments

In [12]:
overwrite = False

# generate custom make str for each test
make = f""
make_extra = f"""pdk={PDK}"""
if PDK == 'intech22': make_extra += f" PDK_CONF=experiments/intech22.yml"
for t,td in tests_dict.items():
    cfg = str(td['root']/'config.yml')
    td['make'] = f"design={td['design']} {make_extra} DESIGN_CONF={cfg}"

# build
build_dpaths = {td['obj_dpath']: t for t,td in tests_dict.items()} # run build once per build dir (not once per test)
for bd,t in build_dpaths.items():
    if overwrite or not bd.exists():
        print(f"make build {tests_dict[t]['make']} -B")
print()

# syn
for bd,t in build_dpaths.items():
    if overwrite or not (bd/"syn-rundir/reports").exists():
        print(f"make syn {tests_dict[t]['make']}")
print()

# sim-rtl
for t in tests_dict:
    fp = tests_dict[t]['root']/'output.fsdb'
    if overwrite or not fp.exists():
        print(f"make redo-sim-rtl {tests_dict[t]['make']}")
print()

# power-rtl
for t in tests_dict:
    if overwrite or not (tests_dict[t]['root']/'power.power.rpt').exists():
        # re-use pre_report_power database if it's already generated (i.e. skip synthesis)
        make_target = "redo-power-rtl args='--only_step report_power'" \
            if (tests_dict[t]['obj_dpath']/'power-rtl-rundir/pre_report_power').exists() else 'power-rtl'
        print(f"make {make_target} {tests_dict[t]['make']}")
print()


## Parse results

In [13]:
def parse_hier_power_rpt(fpath,inst) -> list:
    with fpath.open('r') as f: lines = f.readlines()
    for l in lines:
        words = l.split()
        if l.startswith('Power Unit'):
            assert(words[-1] == 'mW'), f"Wrong power unit in report, {l}"
        if inst == words[-1]:
            return [float(p) for p in words[2:6]]
    return []

def get_duration(fpath) -> float:
    with fpath.open('r') as f: lines = f.readlines()
    time_power = [l.split() for l in lines]
    time_power = [tp for tp in time_power if len(tp) == 2]
    start = float(time_power[0][0])
    end   = float(time_power[-1][0])
    return end-start

time = []
power = list([])
for t in tests_dict:
    fpath = tests_dict[t]['root']/'power.hier.power.rpt'
    power.append(parse_hier_power_rpt(fpath,tests_dict[t]['inst']))
    fpath = tests_dict[t]['root']/'power.profile.png.data'
    time.append(get_duration(fpath))

time = pd.Series(time,   #  ns
                    #  columns=['Duration'],
                     index=tests_dict.keys()) # type: ignore

power = pd.DataFrame(power,   #  mW
                     columns=['Leakage','Internal','Switching','Total'],
                     index=tests_dict.keys()) # type: ignore



energy = power.mul(time,axis=0) / N_ITER # mW * ns = pJ
energy.columns = [c+' (pJ)' for c in energy.columns]
energy.insert(0,'time',time)
energy.insert(0,'test',[i.split('-')[1] for i in energy.index])
energy.insert(0,'design',[i.split('-')[0] for i in energy.index])


energy.to_hdf(PDK+'.h5',key='df',mode='w')
energy


,design,test,time,Leakage (pJ),Internal (pJ),Switching (pJ),Total (pJ)
add8-zero,add8,zero,505.0,0.000002,0.000000,0.000000,0.000002
add8-max_input_switching,add8,max_input_switching,505.0,0.000002,0.616669,11.503799,12.120505
add8-max_output_switching,add8,max_output_switching,505.0,0.000001,0.545569,13.009406,13.555008
add32-zero,add32,zero,505.0,0.000008,0.000000,0.000000,0.000008
add32-max_input_switching,add32,max_input_switching,505.0,0.000008,2.752775,51.149632,53.902488
add32-max_output_switching,add32,max_output_switching,505.0,0.000007,2.288236,52.091659,54.379915


## Notes
What do we want to test?

Inputs:
* 0 -> 0
* 0 -> 11...11
* different activity factors of adds

Designs:
* minimum critical path - can we force the synthesis tool to use faster gates

Flow:
* sim-rtl > power-rtl